In [1]:
# SETUP
from opendp.trans import *
from opendp.meas import *
from opendp.comb import *

from opendp.mod import enable_features

enable_features("floating-point")

In [2]:
# PARSING DATA
data = """
1
2
3
4
5
x
"""[1:]

# Parse lines as integers
preprocessor = (
    make_split_lines() >>
    make_cast(TI=str, TO=int) >>
    make_impute_constant(constant=0)
)
res = preprocessor(data)
print(res)

[1, 2, 3, 4, 5, 0]


In [3]:
# STABILITY RELATION
assert preprocessor.check(d_in=2, d_out=2)

In [4]:
# Parse lines as floats
preprocessor = (
    make_split_lines() >>
    make_cast(TI=str, TO=float) >>
    make_impute_constant(constant=0.)
)
res = preprocessor(data)
print(res)


[1.0, 2.0, 3.0, 4.0, 5.0, 0.0]


In [5]:
# LOADING DATA
data = """
a
b
c
d
e
"""[1:]

# Split file by line
count = make_split_lines() >> make_count(TIA=str)
res = count(data)
print(res)


5


In [6]:
# DATAFRAMES
data = """
ant, 1, 10.0
bat, 2, 20.0
cat, 3, 30.0
foo, 4, 40.0
bar, 5, 50.0
"""[1:]
# Split a dataframe
split_dataframe = make_split_dataframe(separator=",", col_names=[0, 1, 2])
parse = (
    make_parse_column(key=1, T=int) >>
    make_parse_column(key=2, T=float)
)

# Chain splitting and parsing
split_and_parse = split_dataframe >> parse

# look at column 1
res = (split_and_parse >> make_select_column(key=1, T=int))(data)
print(res)

[1, 2, 3, 4, 5]


In [7]:
# SIMPLE OPERATIONS
data = [1., 2., 3., 4., 5.]

# Split and parse lines, chain into sum query
query = (
    make_clamp(bounds=(0., 10.)) >>
    make_bounded_sum(bounds=(0., 10.))
)

In [8]:
# CHECKING PRIVACY
query_laplace = query >> make_base_laplace(scale=1.)
max_user_contribution = 1
budget = 10.
assert query_laplace.check(d_in=max_user_contribution, d_out=budget)
print(query_laplace(data))

11.206906579089827


In [9]:
# TRANSFORMATION REUSE
query_gaussian = query >> make_base_gaussian(scale=1.)
print(query_gaussian(data))


15.55841101155705


In [10]:
# PUTTING IT ALL TOGETHER
data = """
ant, 1, 10.0
bat, 2, 20.0
cat, 3, 30.0
foo, 4, 40.0
bar, 5, 50.0
"""[1:]

parse_dataframe = (
    make_split_dataframe(separator=",", col_names=[0, 1, 2]) >>
    make_parse_column(key=1, T=int) >>
    make_parse_column(key=2, T=float)
)

noisy_sum_col_1 = (
    make_select_column(key=1, T=int) >>
    make_clamp((0, 10)) >>
    make_bounded_sum(bounds=(0, 10)) >>
    make_base_geometric(scale=1.0)
)

noisy_sum_col_2 = (
    make_select_column(key=2, T=float) >>
    make_clamp(bounds=(0., 10.)) >>
    make_bounded_sum(bounds=(0., 10.)) >>
    make_base_laplace(scale=1.0)
)

noisy_count = (
    make_select_column(key=0, T=str) >>
    make_count(TIA=str) >>
    make_base_geometric(scale=1.0)
)

# Compose & chain
composition = make_basic_composition(make_basic_composition(
    noisy_sum_col_1, noisy_sum_col_2), noisy_count)

everything = parse_dataframe >> composition

print(everything(data))

MASKED ERROR: 'AnyObject'
using string fallback
((13, 50.33998462078364), 4)
